*ToDo*

02) pred_window und obs_window ausprobieren.
03) output nochmal genau anschauen.
04) was ist maskiert anschauen - und wie genau?
05) VRAM wenig ausgelastet. Batch size mal mit 320 ausprobieren, aber auch durchziehen.
06) Task pred: scheduler einfügen.
07) Analyse each var (sparsity)
08) loss per var/quality of varis loss (is this already ablation to forecast only one var)
09) Include sepsis definition
10) 

In [1]:
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random 
import joblib
from tqdm import tqdm
tqdm.pandas()
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import roc_auc_score, precision_recall_curve, auc

import torch
import torch.nn as nn
import torch.nn.functional as F
from torchinfo import summary
from torch.optim.lr_scheduler import ReduceLROnPlateau

In [2]:
def set_seed(seed: int) -> None:
    """
    Set the random seed for modules torch, numpy and random.

    :param seed: random seed
    """
    torch.manual_seed(seed)
    np.random.seed(seed)
    random.seed(seed)
    if torch.cuda.is_available() and torch.cuda.device_count() > 0:
        torch.backends.cudnn.deterministic = True
        torch.cuda.manual_seed_all(seed)
set_seed(42)

## Load forecast dataset into matrices.

In [3]:
fore_test_op_dense = joblib.load(open('/home/mitarb/fracarolli/eicu/final/evaluation_scripts+pkl/fore_test_op_dense.pkl', 'rb'))
fore_test_ip_dense = joblib.load(open('/home/mitarb/fracarolli/eicu/final/evaluation_scripts+pkl/fore_test_ip_dense.pkl', 'rb'))
test_input = fore_test_op_dense
test_ip = fore_test_ip_dense

mean_std_dict = joblib.load(open('/home/mitarb/fracarolli/eicu/final/mean_std_dict.pkl', 'rb'))
var_to_ind = dict()
with open("/home/mitarb/fracarolli/eicu/final/var_to_ind.csv") as file:
    for line in file:
        key, value = line.strip().split(",")
        var_to_ind[key]=int(value)


#fore_test_op_triplet = joblib.load(open('/home/mitarb/fracarolli/eicu/final/evaluation_scripts+pkl/fore_test_op_dense.pkl', 'rb'))
#fore_test_ip_triplet = joblib.load(open('fore_test_ip_dense.pkl', 'rb'))

test_input = joblib.load(open("/home/mitarb/hagmann/projects/structures_compositionality/synthetic_test/eicu-full/gold_test.pickle", 'rb'))

# Crossinteractions

# 8 Sofa Analysis IMS Informer TF

In [4]:
d_model, e_layers, d_layers, d_ff = 256, 2, 2, 2048
number_of_epochs = 100
V, D = 98, 17
fore_max_len = 2640
print('loading of joblib completed')



import argparse
parser = argparse.ArgumentParser(description='Autoformer & Transformer family for Time Series Forecasting')

# basic config
parser.add_argument('--is_training', type=int, required=False, default=1, help='status')
parser.add_argument('--train_only', type=bool, required=False, default=False, help='perform training on full input dataset without validation and testing')
parser.add_argument('--model_id', type=str, required=False, default='test', help='model id')
parser.add_argument('--model', type=str, required=False, default='Autoformer',
                    help='model name, options: [Autoformer, Informer, Transformer]')

# data loader
parser.add_argument('--data', type=str, required=False, default='ETTm1', help='dataset type')
parser.add_argument('--root_path', type=str, default='./data/ETT/', help='root path of the data file')
parser.add_argument('--data_path', type=str, default='ETTh1.csv', help='data file')
parser.add_argument('--features', type=str, default='M',
                    help='forecasting task, options:[M, S, MS]; M:multivariate predict multivariate, S:univariate predict univariate, MS:multivariate predict univariate')
parser.add_argument('--target', type=str, default='OT', help='target feature in S or MS task')
parser.add_argument('--freq', type=str, default='h',
                    help='freq for time features encoding, options:[s:secondly, t:minutely, h:hourly, d:daily, b:business days, w:weekly, m:monthly], you can also use more detailed freq like 15min or 3h')
parser.add_argument('--checkpoints', type=str, default='./checkpoints/', help='location of model checkpoints')

# forecasting task
parser.add_argument('--seq_len', type=int, default=24, help='input sequence length')
parser.add_argument('--label_len', type=int, default=0, help='start token length')
parser.add_argument('--pred_len', type=int, default=24, help='prediction sequence length')


# DLinear
parser.add_argument('--individual', action='store_true', default=False, help='DLinear: a linear layer for each variate(channel) individually')
# Formers 
#Michi: bisher nur default=3 zum laufen gebracht
parser.add_argument('--embed_type', type=int, default=3, help='0: default 1: value embedding + temporal embedding + positional embedding 2: value embedding + temporal embedding 3: value embedding + positional embedding 4: value embedding')
parser.add_argument('--enc_in', type=int, default=V*2, help='encoder input size') # DLinear with --individual, use this hyperparameter as the number of channels
parser.add_argument('--dec_in', type=int, default=V, help='decoder input size')
parser.add_argument('--c_out', type=int, default=V, help='output size')
parser.add_argument('--d_model', type=int, default=d_model, help='dimension of model')
parser.add_argument('--n_heads', type=int, default=8, help='num of heads')
parser.add_argument('--e_layers', type=int, default=e_layers, help='num of encoder layers')
parser.add_argument('--d_layers', type=int, default=d_layers, help='num of decoder layers')
parser.add_argument('--d_ff', type=int, default=d_ff, help='dimension of fcn')
parser.add_argument('--moving_avg', type=int, default=25, help='window size of moving average')
parser.add_argument('--factor', type=int, default=1, help='attn factor')
parser.add_argument('--distil', action='store_false',
                    help='whether to use distilling in encoder, using this argument means not using distilling',
                    default=True)
parser.add_argument('--dropout', type=float, default=0.05, help='dropout')
parser.add_argument('--embed', type=str, default='timeF',
                    help='time features encoding, options:[timeF, fixed, learned]')
parser.add_argument('--activation', type=str, default='gelu', help='activation')
parser.add_argument('--output_attention', action='store_true', help='whether to output attention in ecoder')
parser.add_argument('--do_predict', action='store_true', help='whether to predict unseen future data')

# optimization
parser.add_argument('--num_workers', type=int, default=10, help='data loader num workers')
parser.add_argument('--itr', type=int, default=2, help='experiments times')
parser.add_argument('--train_epochs', type=int, default=10, help='train epochs')
parser.add_argument('--batch_size', type=int, default=32, help='batch size of train input data')
parser.add_argument('--patience', type=int, default=3, help='early stopping patience')
parser.add_argument('--learning_rate', type=float, default=0.0001, help='optimizer learning rate')
parser.add_argument('--des', type=str, default='test', help='exp description')
parser.add_argument('--loss', type=str, default='mse', help='loss function')
parser.add_argument('--lradj', type=str, default='type1', help='adjust learning rate')
parser.add_argument('--use_amp', action='store_true', help='use automatic mixed precision training', default=False)

# GPU
parser.add_argument('--use_gpu', type=bool, default=True, help='use gpu')
parser.add_argument('--gpu', type=int, default=0, help='gpu')
parser.add_argument('--use_multi_gpu', action='store_true', help='use multiple gpus', default=False)
parser.add_argument('--devices', type=str, default='0,1,2,3', help='device ids of multile gpus')
parser.add_argument('--test_flop', action='store_true', default=False, help='See utils/tools for usage')

args = parser.parse_args(args=[])
import importlib

loading of joblib completed


# 7 Sofa Analysis DMS Informer

# 10 Sofa Analysis IMS Informer Backprop

In [5]:
import os
import models.InformerAutoregressiveFull as autoformer
importlib.reload(autoformer)
batch_size, lr, patience = 32, 0.0005, 6  # batch_size increased, patience 10 --> 6
d, N, he, dropout = 50, 2, 4, 0.2
model = autoformer.Model(args).cuda()
# Load pretrained weights here.
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
scheduler = ReduceLROnPlateau(optimizer, 'max', patience=3)
for model_path in os.listdir("eicu_full_fixed/"):
    #if not (model_path.endswith("1.model") or model_path.endswith("2.model")): continue
    #if not ("k80" in model_path or "k160" in model_path) or not(model_path.endswith("1.model")): continue
    if not (model_path.endswith(".model")): continue
    print(model_path)
    model.load_state_dict(torch.load("eicu_full_fixed/"+model_path))
    model.eval()
    y_true, y_pred = [], []
    #print(test_input)
    pbar = tqdm(range(0, len(test_input), batch_size))
    loss_list = []
    test_loss = 0
    test_loss_part1 = 0
    test_loss_part2 = 0
    sofa_losses = []
    test_loss_sofa_variables = 0
    accuracy_list = []
    TP, TN, FP, FN = 0, 0, 0, 0
    test_loss_sum_square_total = 0
    test_loss_part1_sum_square_total = 0
    test_loss_part2_sum_square_total = 0
    test_loss_sofa_sum_square_total = 0
    for start in pbar:
        #print(start)
        matrix = torch.tensor(test_input[start:start+batch_size], dtype=torch.float32).cuda()
        input_matrix = matrix[:, :24]

        #torch.Size([32, 24, 129])
        input_mark = torch.arange(0, input_matrix.size(1)).unsqueeze(0).repeat(input_matrix.size(0), 1).cuda()
        #torch.Size([32, 24])
        input_mask = input_matrix[:, :24, V:]
        output_matrix = matrix[:, 24:, :V]
        #torch.Size([32, 24, 129])
        output_mark = torch.arange(0, output_matrix.size(1)).unsqueeze(0).repeat(input_matrix.size(0), 1).cuda()
        #torch.Size([32, 24])
        output_mask = matrix[:, 24:, V:]
        #torch.Size([32, 24, 129])
        output_mark = torch.arange(0, output_matrix.size(1)).unsqueeze(0).repeat(input_matrix.size(0), 1).cuda()
        dec_inp = torch.zeros_like(output_matrix[:, -args.pred_len:, :]).float()
        with torch.no_grad():
            output = model(input_matrix, input_mark, dec_inp, output_mark, trainn=False)
            loss = output_mask[:, -args.pred_len:, :]*(
            output-output_matrix[:, -args.pred_len:, :])**2
        loss_list.extend(loss.sum(axis=-1).mean(axis=-1).detach().cpu().tolist())
        loss_old = loss
        test_loss_sum_square_total += torch.sum(loss_old.sum(axis=-1).mean(axis=-1)**2)
        loss = loss.sum(axis=-1).mean(axis=-1).sum()
        loss_part1 = loss_old[:, :8].sum(axis=-1).mean()
        loss_part2 = loss_old[:, 8:].sum(axis=-1).mean()
        test_loss+=loss
        test_loss_part1+=loss_part1
        test_loss_part2+=loss_part2

        test_loss_part1_sum_square_total += torch.sum(loss_old[:, :8].sum(axis=-1).mean(axis=-1)**2)
        test_loss_part2_sum_square_total += torch.sum(loss_old[:, 8:].sum(axis=-1).mean(axis=-1)**2)
    test_loss_p = test_loss/len(test_input)
    se_loss_p = torch.sqrt(test_loss_sum_square_total / len(test_input) - test_loss_p**2)/len(test_input)
    test_loss_part1_p = test_loss_part1*batch_size/len(test_input)
    test_loss_part2_p = test_loss_part2*batch_size/len(test_input)


    model_name = model_path+'&'
    MSE_string = "{:.3f}".format(test_loss_p.item()) + "$_{[" + "{:.3f}".format(test_loss_p.item()-1.96*se_loss_p.item()) +','+ "{:.3f}".format(test_loss_p.item()+1.96*se_loss_p.item()) + ']}$&'
    print(model_name,MSE_string, sep='',file=open('eicu_full_fixed_other_test.txt','a'))

geca-cut_embedding3h-k40_s42_w1_f4-SILVER_ONLY_True_n_246080000_2_PRETRAINED_MODEL_NONE.model


100%|██████████| 94/94 [00:11<00:00,  8.34it/s]


geca-cut_sadlo-k80_s42_w1_f4-SILVER_ONLY_True_n_246080000_1_PRETRAINED_MODEL_NONE.model


100%|██████████| 94/94 [00:08<00:00, 11.19it/s]


geca-cut_embedding3h-k80_s42_w1_f4-SILVER_ONLY_True_n_246080000_1_PRETRAINED_MODEL_NONE.model


...

100%|██████████| 94/94 [00:08<00:00, 11.20it/s]


geca-cut_embedding3h-k10_s42_w1_f4-SILVER_ONLY_True_n_246080000_1_PRETRAINED_MODEL_NONE.model


100%|██████████| 94/94 [00:08<00:00, 11.28it/s]


geca-cut_embedding3h-k20_s42_w1_f4-SILVER_ONLY_True_n_246080000_1_PRETRAINED_MODEL_NONE.model


100%|██████████| 94/94 [00:08<00:00, 10.68it/s]


In [6]:
import os
import models.InformerAutoregressiveFull as autoformer
importlib.reload(autoformer)
batch_size, lr, patience = 32, 0.0005, 6  # batch_size increased, patience 10 --> 6
d, N, he, dropout = 50, 2, 4, 0.2
model = autoformer.Model(args).cuda()
# Load pretrained weights here.
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
scheduler = ReduceLROnPlateau(optimizer, 'max', patience=3)
for model_path in os.listdir("eicu_full_fixed_finetune/"):
    #if not (model_path.endswith("1.model") or model_path.endswith("2.model")): continue
    #if not ("k80" in model_path or "k160" in model_path) or not(model_path.endswith("1.model")): continue
    if not (model_path.endswith(".model")): continue
    print(model_path)
    model.load_state_dict(torch.load("eicu_full_fixed_finetune/"+model_path))
    model.eval()
    y_true, y_pred = [], []
    #print(test_input)
    pbar = tqdm(range(0, len(test_input), batch_size))
    loss_list = []
    test_loss = 0
    test_loss_part1 = 0
    test_loss_part2 = 0
    sofa_losses = []
    test_loss_sofa_variables = 0
    accuracy_list = []
    TP, TN, FP, FN = 0, 0, 0, 0
    test_loss_sum_square_total = 0
    test_loss_part1_sum_square_total = 0
    test_loss_part2_sum_square_total = 0
    test_loss_sofa_sum_square_total = 0
    for start in pbar:
        #print(start)
        matrix = torch.tensor(test_input[start:start+batch_size], dtype=torch.float32).cuda()
        input_matrix = matrix[:, :24]

        #torch.Size([32, 24, 129])
        input_mark = torch.arange(0, input_matrix.size(1)).unsqueeze(0).repeat(input_matrix.size(0), 1).cuda()
        #torch.Size([32, 24])
        input_mask = input_matrix[:, :24, V:]
        output_matrix = matrix[:, 24:, :V]
        #torch.Size([32, 24, 129])
        output_mark = torch.arange(0, output_matrix.size(1)).unsqueeze(0).repeat(input_matrix.size(0), 1).cuda()
        #torch.Size([32, 24])
        output_mask = matrix[:, 24:, V:]
        #torch.Size([32, 24, 129])
        output_mark = torch.arange(0, output_matrix.size(1)).unsqueeze(0).repeat(input_matrix.size(0), 1).cuda()
        dec_inp = torch.zeros_like(output_matrix[:, -args.pred_len:, :]).float()
        with torch.no_grad():
            output = model(input_matrix, input_mark, dec_inp, output_mark, trainn=False)
            loss = output_mask[:, -args.pred_len:, :]*(
            output-output_matrix[:, -args.pred_len:, :])**2
        loss_list.extend(loss.sum(axis=-1).mean(axis=-1).detach().cpu().tolist())
        loss_old = loss
        test_loss_sum_square_total += torch.sum(loss_old.sum(axis=-1).mean(axis=-1)**2)
        loss = loss.sum(axis=-1).mean(axis=-1).sum()
        loss_part1 = loss_old[:, :8].sum(axis=-1).mean()
        loss_part2 = loss_old[:, 8:].sum(axis=-1).mean()
        test_loss+=loss
        test_loss_part1+=loss_part1
        test_loss_part2+=loss_part2

        test_loss_part1_sum_square_total += torch.sum(loss_old[:, :8].sum(axis=-1).mean(axis=-1)**2)
        test_loss_part2_sum_square_total += torch.sum(loss_old[:, 8:].sum(axis=-1).mean(axis=-1)**2)
    test_loss_p = test_loss/len(test_input)
    se_loss_p = torch.sqrt(test_loss_sum_square_total / len(test_input) - test_loss_p**2)/len(test_input)
    test_loss_part1_p = test_loss_part1*batch_size/len(test_input)
    test_loss_part2_p = test_loss_part2*batch_size/len(test_input)


    model_name = model_path+'&'
    MSE_string = "{:.3f}".format(test_loss_p.item()) + "$_{[" + "{:.3f}".format(test_loss_p.item()-1.96*se_loss_p.item()) +','+ "{:.3f}".format(test_loss_p.item()+1.96*se_loss_p.item()) + ']}$&'
    print(model_name,MSE_string, sep='',file=open('eicu_full_fixed_finetune_other_test.txt','a'))

gold_train-SILVER_ONLY_True_n_246080_1_PRETRAINED_MODEL_geca-cut_sadlo-k10_s41_w1_f4-SILVER_ONLY_True_n_246080000_3_PRETRAINED_MODEL_NONE.model.model


100%|██████████| 94/94 [00:08<00:00, 11.12it/s]


gold_train-SILVER_ONLY_True_n_246080_1_PRETRAINED_MODEL_geca-cut_sadlo-k40_s43_w1_f4-SILVER_ONLY_True_n_246080000_2_PRETRAINED_MODEL_NONE.model.model


100%|██████████| 94/94 [00:08<00:00, 11.02it/s]


gold_train-SILVER_ONLY_True_n_246080_1_PRETRAINED_MODEL_geca-cut_sadlo-k160_s43_w1_f4-SILVER_ONLY_True_n_246080000_3_PRETRAINED_MODEL_NONE.model.model


...

100%|██████████| 94/94 [00:05<00:00, 16.83it/s]


gold_train-SILVER_ONLY_True_n_246080_1_PRETRAINED_MODEL_geca-cut_sadlo-k40_s43_w1_f4-SILVER_ONLY_True_n_246080000_3_PRETRAINED_MODEL_NONE.model.model


100%|██████████| 94/94 [00:05<00:00, 17.34it/s]


gold_train-SILVER_ONLY_True_n_246080_1_PRETRAINED_MODEL_geca-cut_sadlo-k10_s41_w1_f4-SILVER_ONLY_True_n_246080000_1_PRETRAINED_MODEL_NONE.model.model


100%|██████████| 94/94 [00:05<00:00, 17.16it/s]


In [7]:
import os
import models.InformerAutoregressiveFull as autoformer
importlib.reload(autoformer)
batch_size, lr, patience = 32, 0.0005, 6  # batch_size increased, patience 10 --> 6
d, N, he, dropout = 50, 2, 4, 0.2
model = autoformer.Model(args).cuda()
# Load pretrained weights here.
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
scheduler = ReduceLROnPlateau(optimizer, 'max', patience=3)
for model_path in os.listdir("eicu_full_fixed_finetune_higher/"):
    #if not (model_path.endswith("1.model") or model_path.endswith("2.model")): continue
    #if not ("k80" in model_path or "k160" in model_path) or not(model_path.endswith("1.model")): continue
    if not (model_path.endswith(".model")): continue
    print(model_path)
    model.load_state_dict(torch.load("eicu_full_fixed_finetune_higher/"+model_path))
    model.eval()
    y_true, y_pred = [], []
    #print(test_input)
    pbar = tqdm(range(0, len(test_input), batch_size))
    loss_list = []
    test_loss = 0
    test_loss_part1 = 0
    test_loss_part2 = 0
    sofa_losses = []
    test_loss_sofa_variables = 0
    accuracy_list = []
    TP, TN, FP, FN = 0, 0, 0, 0
    test_loss_sum_square_total = 0
    test_loss_part1_sum_square_total = 0
    test_loss_part2_sum_square_total = 0
    test_loss_sofa_sum_square_total = 0
    for start in pbar:
        #print(start)
        matrix = torch.tensor(test_input[start:start+batch_size], dtype=torch.float32).cuda()
        input_matrix = matrix[:, :24]

        #torch.Size([32, 24, 129])
        input_mark = torch.arange(0, input_matrix.size(1)).unsqueeze(0).repeat(input_matrix.size(0), 1).cuda()
        #torch.Size([32, 24])
        input_mask = input_matrix[:, :24, V:]
        output_matrix = matrix[:, 24:, :V]
        #torch.Size([32, 24, 129])
        output_mark = torch.arange(0, output_matrix.size(1)).unsqueeze(0).repeat(input_matrix.size(0), 1).cuda()
        #torch.Size([32, 24])
        output_mask = matrix[:, 24:, V:]
        #torch.Size([32, 24, 129])
        output_mark = torch.arange(0, output_matrix.size(1)).unsqueeze(0).repeat(input_matrix.size(0), 1).cuda()
        dec_inp = torch.zeros_like(output_matrix[:, -args.pred_len:, :]).float()
        with torch.no_grad():
            output = model(input_matrix, input_mark, dec_inp, output_mark, trainn=False)
            loss = output_mask[:, -args.pred_len:, :]*(
            output-output_matrix[:, -args.pred_len:, :])**2
        loss_list.extend(loss.sum(axis=-1).mean(axis=-1).detach().cpu().tolist())
        loss_old = loss
        test_loss_sum_square_total += torch.sum(loss_old.sum(axis=-1).mean(axis=-1)**2)
        loss = loss.sum(axis=-1).mean(axis=-1).sum()
        loss_part1 = loss_old[:, :8].sum(axis=-1).mean()
        loss_part2 = loss_old[:, 8:].sum(axis=-1).mean()
        test_loss+=loss
        test_loss_part1+=loss_part1
        test_loss_part2+=loss_part2

        test_loss_part1_sum_square_total += torch.sum(loss_old[:, :8].sum(axis=-1).mean(axis=-1)**2)
        test_loss_part2_sum_square_total += torch.sum(loss_old[:, 8:].sum(axis=-1).mean(axis=-1)**2)
    test_loss_p = test_loss/len(test_input)
    se_loss_p = torch.sqrt(test_loss_sum_square_total / len(test_input) - test_loss_p**2)/len(test_input)
    test_loss_part1_p = test_loss_part1*batch_size/len(test_input)
    test_loss_part2_p = test_loss_part2*batch_size/len(test_input)


    model_name = model_path+'&'
    MSE_string = "{:.3f}".format(test_loss_p.item()) + "$_{[" + "{:.3f}".format(test_loss_p.item()-1.96*se_loss_p.item()) +','+ "{:.3f}".format(test_loss_p.item()+1.96*se_loss_p.item()) + ']}$&'
    print(model_name,MSE_string, sep='',file=open('eicu_full_fixed_finetune_higher_other_test.txt','a'))

gold_train-SILVER_ONLY_True_n_246080_1_PRETRAINED_MODEL_geca-cut_sadlo-k10_s41_w1_f4-SILVER_ONLY_True_n_246080000_3_PRETRAINED_MODEL_NONE.model.model


100%|██████████| 94/94 [00:05<00:00, 16.22it/s]


gold_train-SILVER_ONLY_True_n_246080_1_PRETRAINED_MODEL_geca-cut_sadlo-k40_s43_w1_f4-SILVER_ONLY_True_n_246080000_2_PRETRAINED_MODEL_NONE.model.model


100%|██████████| 94/94 [00:05<00:00, 16.70it/s]


gold_train-SILVER_ONLY_True_n_246080_1_PRETRAINED_MODEL_geca-cut_sadlo-k160_s43_w1_f4-SILVER_ONLY_True_n_246080000_3_PRETRAINED_MODEL_NONE.model.model


...

100%|██████████| 94/94 [00:05<00:00, 17.59it/s]


gold_train-SILVER_ONLY_True_n_246080_1_PRETRAINED_MODEL_geca-cut_sadlo-k40_s43_w1_f4-SILVER_ONLY_True_n_246080000_3_PRETRAINED_MODEL_NONE.model.model


100%|██████████| 94/94 [00:05<00:00, 16.86it/s]


gold_train-SILVER_ONLY_True_n_246080_1_PRETRAINED_MODEL_geca-cut_sadlo-k10_s41_w1_f4-SILVER_ONLY_True_n_246080000_1_PRETRAINED_MODEL_NONE.model.model


100%|██████████| 94/94 [00:05<00:00, 16.63it/s]


In [8]:
import os
import models.InformerAutoregressiveFull as autoformer
importlib.reload(autoformer)
batch_size, lr, patience = 32, 0.0005, 6  # batch_size increased, patience 10 --> 6
d, N, he, dropout = 50, 2, 4, 0.2
model = autoformer.Model(args).cuda()
# Load pretrained weights here.
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
scheduler = ReduceLROnPlateau(optimizer, 'max', patience=3)
for model_path in os.listdir("eicu_full_fixed_finetune_lower/"):
    #if not (model_path.endswith("1.model") or model_path.endswith("2.model")): continue
    #if not ("k80" in model_path or "k160" in model_path) or not(model_path.endswith("1.model")): continue
    if not (model_path.endswith(".model")): continue
    print(model_path)
    model.load_state_dict(torch.load("eicu_full_fixed_finetune_lower/"+model_path))
    model.eval()
    y_true, y_pred = [], []
    #print(test_input)
    pbar = tqdm(range(0, len(test_input), batch_size))
    loss_list = []
    test_loss = 0
    test_loss_part1 = 0
    test_loss_part2 = 0
    sofa_losses = []
    test_loss_sofa_variables = 0
    accuracy_list = []
    TP, TN, FP, FN = 0, 0, 0, 0
    test_loss_sum_square_total = 0
    test_loss_part1_sum_square_total = 0
    test_loss_part2_sum_square_total = 0
    test_loss_sofa_sum_square_total = 0
    for start in pbar:
        #print(start)
        matrix = torch.tensor(test_input[start:start+batch_size], dtype=torch.float32).cuda()
        input_matrix = matrix[:, :24]

        #torch.Size([32, 24, 129])
        input_mark = torch.arange(0, input_matrix.size(1)).unsqueeze(0).repeat(input_matrix.size(0), 1).cuda()
        #torch.Size([32, 24])
        input_mask = input_matrix[:, :24, V:]
        output_matrix = matrix[:, 24:, :V]
        #torch.Size([32, 24, 129])
        output_mark = torch.arange(0, output_matrix.size(1)).unsqueeze(0).repeat(input_matrix.size(0), 1).cuda()
        #torch.Size([32, 24])
        output_mask = matrix[:, 24:, V:]
        #torch.Size([32, 24, 129])
        output_mark = torch.arange(0, output_matrix.size(1)).unsqueeze(0).repeat(input_matrix.size(0), 1).cuda()
        dec_inp = torch.zeros_like(output_matrix[:, -args.pred_len:, :]).float()
        with torch.no_grad():
            output = model(input_matrix, input_mark, dec_inp, output_mark, trainn=False)
            loss = output_mask[:, -args.pred_len:, :]*(
            output-output_matrix[:, -args.pred_len:, :])**2
        loss_list.extend(loss.sum(axis=-1).mean(axis=-1).detach().cpu().tolist())
        loss_old = loss
        test_loss_sum_square_total += torch.sum(loss_old.sum(axis=-1).mean(axis=-1)**2)
        loss = loss.sum(axis=-1).mean(axis=-1).sum()
        loss_part1 = loss_old[:, :8].sum(axis=-1).mean()
        loss_part2 = loss_old[:, 8:].sum(axis=-1).mean()
        test_loss+=loss
        test_loss_part1+=loss_part1
        test_loss_part2+=loss_part2

        test_loss_part1_sum_square_total += torch.sum(loss_old[:, :8].sum(axis=-1).mean(axis=-1)**2)
        test_loss_part2_sum_square_total += torch.sum(loss_old[:, 8:].sum(axis=-1).mean(axis=-1)**2)
    test_loss_p = test_loss/len(test_input)
    se_loss_p = torch.sqrt(test_loss_sum_square_total / len(test_input) - test_loss_p**2)/len(test_input)
    test_loss_part1_p = test_loss_part1*batch_size/len(test_input)
    test_loss_part2_p = test_loss_part2*batch_size/len(test_input)


    model_name = model_path+'&'
    MSE_string = "{:.3f}".format(test_loss_p.item()) + "$_{[" + "{:.3f}".format(test_loss_p.item()-1.96*se_loss_p.item()) +','+ "{:.3f}".format(test_loss_p.item()+1.96*se_loss_p.item()) + ']}$&'
    print(model_name,MSE_string, sep='',file=open('eicu_full_fixed_finetune_lower_other_test.txt','a'))

gold_train-SILVER_ONLY_True_n_246080_1_PRETRAINED_MODEL_geca-cut_sadlo-k10_s41_w1_f4-SILVER_ONLY_True_n_246080000_3_PRETRAINED_MODEL_NONE.model.model


100%|██████████| 94/94 [00:05<00:00, 16.32it/s]


gold_train-SILVER_ONLY_True_n_246080_1_PRETRAINED_MODEL_geca-cut_sadlo-k40_s43_w1_f4-SILVER_ONLY_True_n_246080000_2_PRETRAINED_MODEL_NONE.model.model


100%|██████████| 94/94 [00:05<00:00, 16.50it/s]


gold_train-SILVER_ONLY_True_n_246080_1_PRETRAINED_MODEL_geca-cut_sadlo-k160_s43_w1_f4-SILVER_ONLY_True_n_246080000_3_PRETRAINED_MODEL_NONE.model.model


...

100%|██████████| 94/94 [00:05<00:00, 17.01it/s]


gold_train-SILVER_ONLY_True_n_246080_1_PRETRAINED_MODEL_geca-cut_sadlo-k40_s43_w1_f4-SILVER_ONLY_True_n_246080000_3_PRETRAINED_MODEL_NONE.model.model


100%|██████████| 94/94 [00:05<00:00, 16.85it/s]


gold_train-SILVER_ONLY_True_n_246080_1_PRETRAINED_MODEL_geca-cut_sadlo-k10_s41_w1_f4-SILVER_ONLY_True_n_246080000_1_PRETRAINED_MODEL_NONE.model.model


100%|██████████| 94/94 [00:05<00:00, 17.05it/s]
